In [6]:
import cv2 as cv
import os
from os.path import exists
import numpy as np
import mediapipe as mp

drowsiness_level = np.array(['0', '5', '10'])

Create folders for extracted values

In [7]:
DATA_PATH = os.path.join('Extracted_Values')

for number in range(1,61):
    for level in drowsiness_level:
        try:
            os.makedirs(os.path.join(DATA_PATH, str(number), str(level)))
        except:
            pass


In order to solve openCV problem use version 4.5.5.62

In [3]:
def extractKeypoints(result):
    if result.multi_face_landmarks:
        for face_detected in result.multi_face_landmarks:
            face = np.array([[res.x, res.y, res.z] for res in face_detected.landmark]).flatten()
    else:
        face = np.zeros(1434)

    return face

Extract face features from video

In [ ]:
from sys import api_version


mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils  
mp_drawing_styles = mp.solutions.drawing_styles

folder_numbers = range(1,3)
video_extension_lowercase = ".mov"
video_extension_uppercase = ".MOV"


for number in folder_numbers:
    for level in drowsiness_level:
        video_path = "Dataset/" + str(number) + "/" + str(level) + video_extension_lowercase
        if exists(video_path) == False:
            video_path = "./Dataset/" + str(number) + "/" + str(level) + video_extension_uppercase

        cap = cv.VideoCapture(video_path)
        frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))  # Count the number of frames in each video


        with mp_face_mesh.FaceMesh(
            static_image_mode=False,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.6,
            min_tracking_confidence = 0.6) as mesh:

            for frame_number in range(frame_count):
                ret, frame = cap.read()
                
                frame = cv.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv.INTER_LINEAR)    

                image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)  
                image.flags.writeable = False 
                results = mesh.process(image)
                image.flags.writeable = True
                image = cv.cvtColor(image, cv.COLOR_RGB2BGR)  

                if results.multi_face_landmarks:
                    for face_landmarks in results.multi_face_landmarks:
                        mp_drawing.draw_landmarks(
                            image=image,
                            landmark_list=face_landmarks,
                            connections=mp_face_mesh.FACEMESH_TESSELATION,
                            landmark_drawing_spec=None,
                            connection_drawing_spec=mp_drawing_styles
                            .get_default_face_mesh_tesselation_style())
                        mp_drawing.draw_landmarks(
                            image=image,
                            landmark_list=face_landmarks,
                            connections=mp_face_mesh.FACEMESH_CONTOURS,
                            landmark_drawing_spec=None,
                            connection_drawing_spec=mp_drawing_styles
                            .get_default_face_mesh_contours_style())
                        mp_drawing.draw_landmarks(
                            image=image,
                            landmark_list=face_landmarks,
                            connections=mp_face_mesh.FACEMESH_IRISES,
                            landmark_drawing_spec=None,
                            connection_drawing_spec=mp_drawing_styles
                            .get_default_face_mesh_iris_connections_style())

                landmarks = extractKeypoints(results)
                npyPath = os.path.join("./Extracted_Values/", str(number), str(level), str(frame_number))
                np.save(npyPath, landmarks)

                # cv.imshow('Video', image)
                # if cv.waitKey(1) == ord('q'):
                #     break

            cap.release()
            # cv.destroyAllWindows()